### Map results back to shapefile
The functions below prepare the data for the process of mapping the results back to their original shapefiles by implementing a "table join" between the outputs .csv and the attribute table of the shapefile. This will allow the visualization of accessibility **before** optimization.

In [131]:
import pandas as pd
import os, sys
sys.path.append(r'../../../GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
from shapely.wkt import loads
import numpy as np

networkx version: 2.2 
osmnx version: 0.9 


In [132]:
#define the paths to the origins, destinations and .pickle file
pth = r'../../../../lima_optimization_output'
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32718'}

In [133]:
# read  file
OD_name = r'saved_OD.csv'
OD = pd.read_csv(os.path.join(pth, OD_name), sep=',')
OD = OD.rename(columns = {'Unnamed: 0':'NN'})

In [134]:
#preview
OD[:3]

,NN,6048,2048,6691,4154,4198,4647,4233,3914,2959,175,3409,367,1556,917,4919,474,6107
0,6147,1968.655016,1020.721567,363.676689,819.749517,1322.002788,1578.076932,1803.247252,806.455001,2049.990821,1517.705535,1879.396717,803.156991,1542.402922,1657.102815,583.090841,1736.154601,1584.512721
1,2052,525.817013,558.842693,1633.650766,1591.881656,274.594031,839.743363,1166.553316,1578.626774,872.524426,1600.311304,1019.720100,1744.228690,875.417373,694.618499,1785.150976,666.975509,544.361984
2,3,1330.771618,448.775456,1230.200991,1380.165021,873.074022,1498.688595,1962.518259,1307.025267,1668.335516,2097.397326,1815.601711,1546.804731,1534.362605,1386.775967,1520.879916,1471.197652,1236.519452


In [135]:
OD_ = OD.set_index('NN')
#replacing infinity and negative infinity with not a number
OD = OD_.replace([np.inf, -np.inf], np.nan)
#adding a column that has the minimum time to reach each origin's nearest facility
OD['Minimum'] = OD.loc[:,:].min(axis=1)

In [136]:
OD[:3]

,6048,2048,6691,4154,4198,4647,4233,3914,2959,175,3409,367,1556,917,4919,474,6107,Minimum
NN,,,,,,,,,,,,,,,,,,
6147,1968.655016,1020.721567,363.676689,819.749517,1322.002788,1578.076932,1803.247252,806.455001,2049.990821,1517.705535,1879.396717,803.156991,1542.402922,1657.102815,583.090841,1736.154601,1584.512721,363.676689
2052,525.817013,558.842693,1633.650766,1591.881656,274.594031,839.743363,1166.553316,1578.626774,872.524426,1600.311304,1019.720100,1744.228690,875.417373,694.618499,1785.150976,666.975509,544.361984,274.594031
3,1330.771618,448.775456,1230.200991,1380.165021,873.074022,1498.688595,1962.518259,1307.025267,1668.335516,2097.397326,1815.601711,1546.804731,1534.362605,1386.775967,1520.879916,1471.197652,1236.519452,448.775456


### PREPARE ORIGIN FILE 
set index to nearest node to match on the travel time from that node to nearest destination (OD['Minimum'])

In [137]:
orig = pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
#orig = orig.rename({'ID':'OBJECTID'}, axis = 1) # rename ID column to 'OBJECTID'
orig = orig.set_index('NN') # used for matching on the OD matrix
orig[:3]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist
NN,,,,,,,,,,,
6528,0,1,3.0,0.003205,2.860800e-07,0,1078.0,-12.24839,-76.91749,POINT (-76.91749 -12.24839),21.347245
5270,1,2,3.0,0.008010,2.565700e-06,1,2374.0,-12.24319,-76.92767,POINT (-76.92766999999999 -12.24319),37.506920
1921,2,3,3.0,0.010808,2.141710e-06,2,367.0,-12.24444,-76.91685,POINT (-76.91685 -12.24444),6.260416


In [138]:
#join works on the index as default 
join = orig.join(OD)
join

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,...,2959,175,3409,367,1556,917,4919,474,6107,Minimum
NN,,,,,,,,,,,,,,,,,,,,,
3,633,634,4.0,0.028739,8.435590e-06,633,1458.0,-12.23372,-76.94772,POINT (-76.94771999999999 -12.23372),...,1668.335516,2097.397326,1815.601711,1546.804731,1534.362605,1386.775967,1520.879916,1471.197652,1236.519452,448.775456
21,488,489,3.0,0.005872,1.814450e-06,488,2232.0,-12.19822,-76.96106,POINT (-76.96106 -12.19822),...,509.757316,1102.641524,680.047647,1902.078238,1525.906486,1304.225116,1943.000525,1031.775449,1073.514728,509.757316
32,89,90,3.0,0.005751,1.013350e-06,89,2041.0,-12.23260,-76.91658,POINT (-76.91658000000001 -12.2326),...,1731.880990,1153.011713,1514.702896,278.204115,1177.709101,1362.446221,295.462617,1371.460779,1511.728358,278.204115
82,549,550,3.0,0.003212,6.469200e-07,549,1508.0,-12.19633,-76.94200,POINT (-76.94199999999999 -12.19633),...,683.230267,422.878004,466.052172,1037.260113,715.719204,826.848555,1078.182399,777.169968,973.267778,389.902708
84,212,213,3.0,0.003279,6.724400e-07,212,1610.0,-12.22342,-76.94540,POINT (-76.94540000000001 -12.22342),...,991.984789,1715.993081,1138.474966,1437.392600,880.688869,699.889995,1478.314887,785.389131,549.633480,183.718794
99,422,423,3.0,0.004647,1.284970e-06,422,1295.0,-12.20459,-76.95295,POINT (-76.95295 -12.20459),...,90.873706,901.329151,280.090546,1515.711259,1047.276286,825.333230,1556.633546,553.145249,603.017870,90.873706
106,112,113,3.0,0.009651,3.008850e-06,112,1216.0,-12.23132,-76.93962,POINT (-76.93961999999999 -12.23132),...,1355.122106,1759.288755,1501.480910,1102.688750,1203.718357,1056.917905,1189.459920,1147.713915,912.770797,215.854603
114,307,308,3.0,0.013462,4.455160e-06,307,824.0,-12.21750,-76.93547,POINT (-76.93547 -12.2175),...,1135.456665,1273.050393,995.954493,992.486265,437.746181,328.649809,1033.408552,614.931277,547.879949,328.649809
124,581,582,3.0,0.008233,2.820590e-06,581,440.0,-12.19100,-76.95468,POINT (-76.95468000000001 -12.191),...,691.278465,786.803430,651.092484,1586.240144,1354.311179,1242.753618,1627.162431,1005.405667,1097.740446,650.977439


In [139]:
# add on walktime (kph) for time computation from each origin node's original location to its nearest node on the graph
walkspeed = 4
# generate walktime in seconds to 'NN'
join['walktime'] = join['NN_dist'] / 1000 / walkspeed * 3600
# total_time in minutes
join['total_time_min'] = (join['walktime'] + join['Minimum']) / 60
###   OR   ###
# compute total distance: NN_dist + dist
# this is assuming that the 'Minimum' values are distances instead of time
#join['total_distance'] = (join['NN_dist'] + join['Minimum'])

In [140]:
#read in the original origins (neighborhood polygons) in shapefile format
fp = r'../../Peru_Lima_Optimization/test_data'
shpfil = gpd.read_file(os.path.join(fp, 'VillaElSalvador_urban.shp'))

In [141]:
# Ensure both shapefile AND origin file have a non-duplicate index to enable matching of results on to the shapefile
if shpfil.index.name != 'OBJECTID':
    shpfil = shpfil.set_index('OBJECTID')
#join.index.names = ['OBJECTID']

In [142]:
shpfil[:10]

,NC_CLASS,Centr_Long,Centr_Lat,Shape_Leng,Shape_Area,geometry
OBJECTID,,,,,,
1,3.0,291411.942758,8.645182e+06,0.003205,2.860805e-07,"POLYGON ((-76.9167549641702 -12.2482398349004,..."
2,3.0,290309.820276,8.645806e+06,0.008010,2.565702e-06,POLYGON ((-76.92622229552251 -12.2430487070321...
3,3.0,291520.979384,8.645699e+06,0.010808,2.141712e-06,POLYGON ((-76.91690291343866 -12.2445463561293...
4,3.0,290041.875910,8.645860e+06,0.004083,7.910232e-07,POLYGON ((-76.93050875577444 -12.2420243675305...
5,3.0,290264.651133,8.645883e+06,0.006058,1.065815e-06,POLYGON ((-76.92877667140624 -12.2431240549311...
6,3.0,291484.441150,8.645801e+06,0.011915,4.861858e-06,POLYGON ((-76.9146477078254 -12.24268260471837...
7,3.0,289929.100207,8.645739e+06,0.020647,7.236286e-06,POLYGON ((-76.93224834138783 -12.2423044335027...
8,3.0,289930.248147,8.645951e+06,0.003497,5.525647e-07,POLYGON ((-76.93061953246541 -12.2418276336383...
9,3.0,289656.869602,8.645958e+06,0.004676,1.196259e-06,POLYGON ((-76.93363256978722 -12.2423731507003...


In [146]:
# testing
#join
#join.loc[join['O_ID'] < 10]

In [147]:
# set index to O_ID so that you can add the 'total_time' column to shpfil GPD, which also has an index of the Object ID
join = join.set_index('O_ID')

In [148]:
# Prep output shapefile, match on visualization column, save file
shpfil['total_time_min'] = join['total_time_min']

In [149]:
shpfil[:5]

,NC_CLASS,Centr_Long,Centr_Lat,Shape_Leng,Shape_Area,geometry,total_time_min
OBJECTID,,,,,,,
1,3.0,291411.942758,8.645182e+06,0.003205,2.860805e-07,"POLYGON ((-76.9167549641702 -12.2482398349004,...",8.490326
2,3.0,290309.820276,8.645806e+06,0.008010,2.565702e-06,POLYGON ((-76.92622229552251 -12.2430487070321...,7.143033
3,3.0,291520.979384,8.645699e+06,0.010808,2.141712e-06,POLYGON ((-76.91690291343866 -12.2445463561293...,6.274351
4,3.0,290041.875910,8.645860e+06,0.004083,7.910232e-07,POLYGON ((-76.93050875577444 -12.2420243675305...,8.206695
5,3.0,290264.651133,8.645883e+06,0.006058,1.065815e-06,POLYGON ((-76.92877667140624 -12.2431240549311...,7.924854


In [150]:
shpfil = shpfil.to_crs({'init':'epsg:4326'})
shpfil.to_file(os.path.join(pth, 'VES_origins_w_time_pre_optimization.shp'), driver = 'ESRI Shapefile')

In [83]:
# write the road file as a shapefile
from shapely.wkt import loads
edges = pd.read_csv(os.path.join(pth, 'biggest_subg_edges.csv'))
edges['geometry'] = edges['geometry'].apply(loads)
edges_gdf = gpd.GeoDataFrame(edges, geometry = 'geometry', crs = {'init':'epsg:4326'})
edges_gdf.to_file(os.path.join(pth, 'VES_roads.shp'), driver = 'ESRI Shapefile')